In [4]:
import os
import re
import time
import pandas as pd
import pdfplumber
from PyPDF2 import PdfReader
from tabulate import tabulate
from sqlalchemy import create_engine
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

In [5]:
# Daftar bulan untuk pencocokan
bulan_list_id = [
    "Januari", "Februari", "Maret", "April", "Mei", "Juni",
    "Juli", "Agustus", "September", "Oktober", "November", "Desember"
]

bulan_list_en = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]

# Fungsi mengambil teks dari halaman atau indeks pertama file pdf
def extract_first_pages_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        # Mengambil teks dari halaman pertama
        first_page = pdf.pages[0]
        text = first_page.extract_text()
    return text


# Fungsi mengambil teks dari halaman 1 file pdf
def extract_text_from_pages(pdf_path, start_page=1, end_page=None):
    try:
        reader = PdfReader(pdf_path)
        text_content = ""
        total_pages = len(reader.pages)

        if end_page is None or end_page > total_pages:
            end_page = total_pages

        # Mengekstrak teks dari halaman yang ditentukan
        for page_num in range(start_page - 1, end_page):
            page = reader.pages[page_num]
            text_content += page.extract_text() + "\n"
        return text_content
    except Exception as e:
        print(f"Error reading PDF {pdf_path}: {str(e)}")
        return None


# Fungsi untuk mengekstrak tabel
def extract_table_data(text_content, filename):
    # Membagi konten menjadi baris
    lines = text_content.split('\n')

    # Inisialisasi list untuk menyimpan hasil ekstraksi
    items = []
    values = []
    notes = []
    file_sources = []

    # Regular expression
    # Untuk mengekstrak angka
    number_pattern = r'\(?[\d,.]+\)?' 

    # Mengekstrak pattern note (2v,2b  18, 5c)
    notes_pattern_1 = r'\d+[a-zA-Z](?:,\s?[a-zA-Z\d]+)*' 

    # Mengekstrak pattern note alternatif (23,25  29)
    notes_pattern_2 = r'\b\d{1,2}(?:,\s?\d{1,2})*\b'

    for line in lines:
        line = line.strip()

        # Mengindentifikasi komponen
        number_match = re.search(number_pattern, line)
        notes_match_1 = re.search(notes_pattern_1, line)
        notes_match_2 = re.search(notes_pattern_2, line)

        if number_match:
            # Mengekstrak 'item' dan menghilangkan 'numbers' dan 'notes' dari item
            item = re.sub(number_pattern, '', line)  
            item = re.sub(notes_pattern_1, '', item)  
            item = re.sub(notes_pattern_2, '', item) 
            item = re.sub(r'\s+', ' ', item).strip() 

            # Menghilangkan tanda kurung kosong () dan teks setelahnya
            item = re.sub(r'\(\s*\).*', '', item).strip()

            # Menghilangkan huruf tunggal atau ganda (v  fv) dan teks setelahnya
            item = re.sub(r'\s[a-zA-Z]{1,2}\s.*', '', item).strip()

            # Menghilangkan teks setelahnya apabila dimulai dengan huruf besar (bukan semua kapital)
            # item = re.sub(r'\s([A-Z][a-z][a-zA-Z]*)\b.*', '', item).strip()

            # Mengekstrak value berdasarkan 'number_pattern'
            numbers = re.findall(number_pattern, line)
            if len(numbers) > 1:
                value = numbers[1]
            elif len(numbers) == 1:
                value = numbers[0]
            else:
                value = " "

            # Menggunakan notes_pattern 1 jika sesuai
            if notes_match_1:
                note = notes_match_1.group()
            # Hanya menggunakan notes_patern 2 jika notes_pattern 1 tidak sesuai
            elif notes_match_2 and not notes_match_1:
                note = notes_match_2.group()
            else:
                note = ""  # Default case if no notes are found

            # Modifikasi values
            if value.startswith('(') and value.endswith(')') or value.startswith('('):
                # Mengubah tanda dalam kurung menjadi negatif
                value = '-' + value[1:-1].replace('.', '').replace(',', '')
            else:
                # Menghilangkan tanda . dan ,
                value = value.replace('.', '').replace(',', '')

            
            # Append value dan note ke masing-masing list
            values.append(value)
            notes.append(note)

            # Hanya menyertakan semua komponen dan tidak dimulai dengan 'total'
            if item and not item.lower().startswith('total'):
                items.append(item)
                file_sources.append(filename)  # Add filename to track source

            # Memastikan list memiliki panjang yang sama
            while len(items) < len(values):
                items.append("")
            while len(notes) < len(values):
                notes.append("")
            while len(file_sources) < len(values):
                file_sources.append("")

    # Membuat dataframe
    df = pd.DataFrame({
        'Item': items,
        'Notes': notes,
        'Value': values,
        'Source_File': file_sources
    })
    
    return df


# Fungsi mencari nama perusahaan, bulan, dan tahun
def find_company_and_month_year(text):
    # Pattern nama perusahaan
    company_pattern = re.compile(r"(PT\s[\w\s]+,\s?Tbk|PT\s[\w\s]+Tbk)", re.IGNORECASE)
    company_match = company_pattern.search(text)
    
    # Mencocokkan bulan dan tahun
    months_years = []
    for bulan in bulan_list_id + bulan_list_en:
        pattern = re.compile(rf"(\b{bulan}\b)\s(\d{{4}})", re.IGNORECASE)
        
        # Mencari kecocokan dengan pattern regex
        matches = pattern.findall(text)
        
        for match in matches:
            month, year = match
            months_years.append((month.strip(), int(year)))

    # Mengekstrak nama perusahaan
    company_name = company_match.group(1) if company_match else "Tidak ditemukan"
    
    return company_name, months_years


# Fungsi mendapatkan bulan dan tahun terakhir
def get_latest_month_and_year(months_years):
    if not months_years:
        return None, None
    # Tahun terakhir
    latest_year = max(months_years, key=lambda x: x[1])[1]
    # Bulan terakhir
    latest_month = next(month for month, year in months_years if year == latest_year)
    
    return latest_month, latest_year


# Fungsi mengkonversi bulan menjadi kuartal
def convert_month_to_quarter(month):
    # Normalisasi nama bulan
    month = month.strip().capitalize()
    
    # List tiap kuartal
    q1 = ["Januari", "January", "Februari", "February", "Maret", "March"]
    q2 = ["April", "Mei", "May", "Juni", "June"]
    q3 = ["Juli", "July", "Agustus", "August", "September"]
    q4 = ["Oktober", "October", "November", "Desember", "December"]
    
    # Kondisi kuartal
    if month in q1:
        return "Kuartal I / First Quarter"
    elif month in q2:
        return "Kuartal II / Second Quarter"
    elif month in q3:
        return "Kuartal III / Third Quarter"
    elif month in q4:
        return "Tahunan / Annual"
    else:
        return "Unknown"


# Fungsi filer dataframe
def filter_dataframe(df):
    if df is None:
        return None
    
    # Fungsi untuk mengecek apakah value valid
    def is_valid_value(value):
        # Konversi value ke tipe string
        str_value = str(value).strip()
        
        # Pattern yang ingin dihilangkan
        unwanted_patterns = [
            r'^[\.]+$',  # hanya dot
            r'^2024$',   # hanya 2024
            r'^2023$',   # hanya 2023
            r'^[\.]{5,}$'  # 5 atau lebih dot
        ]
        
        # Mengecek apakah value sesuai atau cocok dengan unwanted_patterns
        for pattern in unwanted_patterns:
            if re.match(pattern, str_value):
                return False
        
        # Mengecek apakah value terdiri dari minimal 1 digit
        digit_count = len(re.findall(r'\d', str_value))
        
        return digit_count >= 1
    
    # Filter dataframe
    filtered_df = df[df['Value'].apply(is_valid_value)]
    
    # Print banyak baris yang dihilangkan
    original_rows = len(df)
    filtered_rows = len(filtered_df)
    removed_rows = original_rows - filtered_rows
    print(f"Filtered out {removed_rows} rows (from {original_rows} to {filtered_rows} rows)")
    
    return filtered_df


# Fungsi untuk memproses folder
def process_pdf_folder(folder_path, table_start_page=1, table_end_page=8):
    table_dfs = []
    info_dfs = []
    # Mengambil path dari folder
    folder_path = os.path.normpath(folder_path)
    # Menentukan file dengan format .pdf
    pdf_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.pdf')]

    if not pdf_files:
        print(f"No PDF files found in {folder_path}")
        return None

    for pdf_file in pdf_files:
        print(f"Processing {pdf_file}...")
        pdf_path = os.path.join(folder_path, pdf_file)

        # Ekstrak teks dari halaman pertama untuk nama perusahaan dan kuartal
        first_page_text = extract_first_pages_pdf(pdf_path)
        if first_page_text:
            company_name, months_years = find_company_and_month_year(first_page_text)
            latest_month, latest_year = get_latest_month_and_year(months_years)

            if latest_month and latest_year:
                quarter = convert_month_to_quarter(latest_month)
                info_df = pd.DataFrame({
                    'Nama Perusahaan': [company_name],
                    'Kuartal': [quarter],
                    'Tahun': [latest_year],
                    'Source_File': [pdf_file]
                })
                info_dfs.append(info_df)

        # Ekstrak teks dari halaman 1–8 untuk tabel
        table_text = extract_text_from_pages(pdf_path, table_start_page, table_end_page)
        if table_text:
            table_df = extract_table_data(table_text, pdf_file)
            if not table_df.empty:
                table_dfs.append(table_df)

    # Menggabungkan dataframe
    if table_dfs and info_dfs:
        combined_table_df = pd.concat(table_dfs, ignore_index=True)
        combined_info_df = pd.concat(info_dfs, ignore_index=True)

        # Menggabungkan dataframe berdasarkan kolom Source_File
        merged_pdf = pd.merge(combined_info_df, combined_table_df, on='Source_File', how='left')
        # Drop kolom Source_File
        merged_pdf = merged_pdf.drop(columns=['Source_File'])
        # Menyelaraskan value pada kolom Nama Perusahaan
        merged_pdf['Nama Perusahaan'] = merged_pdf['Nama Perusahaan'].str.upper().str.replace(', TBK', ' TBK')
        # Menghilangkan spasi pada kolom Value
        merged_pdf['Value'] = merged_pdf['Value'].str.replace(' ', '', regex=True)

        filtered_pdf = filter_dataframe(merged_pdf)
        return filtered_pdf
    else:
        print("No data was extracted from any PDF files")
        return None


# Fungsi mendapatkan merged_pdf
def get_merged_pdf_data(folder_path):
    merged_pdf = process_pdf_folder(folder_path)
    
    if merged_pdf is not None:
        print("\nMerged and Filtered PDF Data:")
        print(tabulate(merged_pdf, headers='keys', tablefmt='grid', showindex=False))
        return merged_pdf
    else:
        print("Tidak ada data yang berhasil diekstrak.")
        return None


# Fungsi export dataframe ke database MySQL
def export_to_mysql(merged_df, table_name='tb_pdf', 
                    mysql_user='-', 
                    mysql_password='-', 
                    mysql_host='-', 
                    mysql_port='-', 
                    db_name='-'):
    try:
        # Mapping nama kolom sesuai dengan database
        column_mapping = {
            'Nama Perusahaan': 'nama_perusahaan_pdf',
            'Kuartal': 'kuartal_pdf',
            'Tahun': 'tahun_pdf',
            'Item': 'item_pdf',
            'Notes': 'notes_pdf',
            'Value': 'value_pdf',
        }
        
        # Mengganti nama kolom sesuai mapping
        merged_df = merged_df.rename(columns=column_mapping)
        # SQLAlchemy engine
        engine = create_engine(f'mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/{db_name}')
        # Export dataframe ke MySQL
        merged_df.to_sql(table_name, con=engine, if_exists='append', index=False)
        
        print(f"\nData successfully exported to MySQL database '{db_name}', table '{table_name}'.")
    except Exception as e:
        print(f"Error exporting to MySQL: {e}")


# Class untuk menangani event file baru
class MyHandler(FileSystemEventHandler):
    def __init__(self, folder_path):
        self.folder_path = os.path.normpath(folder_path)

    def on_created(self, event):
        if event.is_directory:
            return  # Skip folder
        
        # Normalize the file path
        event_src_path = os.path.normpath(event.src_path)
        
        if event_src_path.lower().endswith('.pdf'): 
            processed_file = event_src_path + '.processed'
            if not os.path.exists(processed_file):
                print(f"\nFile baru ditemukan: {event_src_path}")
                process_and_export(event_src_path)
            else:
                print(f"File {event_src_path} sudah diproses sebelumnya, melewatkan...")


# Fungsi untuk memproses dan mengekspor data dari file PDF
def process_and_export(pdf_path):
    # Normalisasi pdf_path
    pdf_path = os.path.normpath(pdf_path)
    
    # Ambil direktori dari pdf_path
    pdf_folder = os.path.dirname(pdf_path)
    
    # Mengambil data dari PDF
    merged_pdf = get_merged_pdf_data(pdf_folder)
    
    if merged_pdf is not None:
        # Export ke MySQL
        export_to_mysql(
            merged_pdf,
            table_name='tb_pdf', 
            mysql_user='root', 
            mysql_password='Dika007!#', 
            db_name='db_calk' 
        )
        # Tandai file sudah diproses
        processed_file = pdf_path + '.processed'
        with open(processed_file, 'w') as f:
            f.write('Processed') 
        print(f"File {pdf_path} berhasil diproses dan diekspor ke database.")
    else:
        print(f"Gagal memproses file {pdf_path}")


# Fungsi untuk memproses semua file yang sudah ada
def process_existing_files(folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_path.lower().endswith('.pdf'): 
            processed_file = file_path + '.processed'
            if not os.path.exists(processed_file): 
                print(f"\nMemproses file yang sudah ada: {file_path}")
                process_and_export(file_path)
            else:
                print(f"File {file_path} sudah diproses sebelumnya, melewatkan...")

# Membuat observer untuk memantau folder
def start_monitoring(folder_path):
    event_handler = MyHandler(folder_path)
    observer = Observer()
    observer.schedule(event_handler, folder_path, recursive=False)
    
    # Proses file yang sudah ada di folder
    process_existing_files(folder_path)
    
    # Mulai memantau folder
    observer.start()
    print(f"Memantau folder: {folder_path} untuk file baru...")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
        print("\nPemantauan dihentikan.")
    observer.join()

# Main function
if __name__ == "__main__":
    folder_path = 'C:/TI-Dwika/Semester 7/Pangkalan Data/file_pdf'
    start_monitoring(folder_path)


Memproses file yang sudah ada: C:/TI-Dwika/Semester 7/Pangkalan Data/file_pdf\AGAR TW 2 KONSOL 2024.pdf
Processing AGAR TW 2 KONSOL 2024.pdf...
Processing FKS Food Sejahtera TBK billingual 31 Maret 2024.pdf...
Processing Lapkeu ABBA 31 Mar 2024.pdf...
Processing LKA PT Samcro Hyosung Adilestari Tbk Maret 2024.pdf...
Filtered out 172 rows (from 512 to 340 rows)

Merged and Filtered PDF Data:
+----------------------------------+-----------------------------+---------+--------------------------------------------------------------------------+----------------+---------------+
| Nama Perusahaan                  | Kuartal                     |   Tahun | Item                                                                     | Notes          | Value         |
+==================================+=============================+=========+==========================================================================+================+===============+
| PT ASIA SEJAHTERA MINA TBK       | Kuartal II / 